# Assignment 3 -- Policy Gradients
## Instructions:
* This is an individual assignment. You are not allowed to discuss the problems with other students.
* Part of this assignment will be autograded by gradescope. You can use it as immediate feedback to improve your answers. You can resubmit as many times as you want.
* All your solution, code, analysis, graphs, explanations should be done in this same notebook.
* Please make sure to execute all the cells before you submit the notebook to the gradescope. You will not get points for the plots if they are not generated already.
* If you have questions regarding the assignment, you can ask for clarifications in Piazza. You should use the corresponding tag for this assignment.

This assignment has 4 parts. The goals of these parts are:
- **Part 1**: Implementing a parameterized (neural network) policy with PyTorch
- **Part 2**: Understanding the REINFORCE algorithm
- **Part 3**: Extending the REINFORCE algorithm with a baseline
- **Part 4**: Understanding Actor-Critic


### Loading Dependencies

In [ ]:
!pip install otter-grader
!pip install gym[classic_control]
!git clone https://github.com/chandar-lab/INF8250e-assignments-public.git public

In [ ]:
import otter
grader = otter.Notebook(colab=True, tests_dir='./public/assignment_3_python/tests')

In [ ]:
import gym
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
import torch.distributions as torchdist
import torch.nn as nn
import torch.nn.functional as F
import warnings
import functools
import os

from gym.envs.classic_control import CartPoleEnv
from tqdm import tqdm

warnings.filterwarnings('ignore')

In [ ]:
GRADESCOPE_ENV_VAR = "RUNNING_IN_GRADESCOPE"

def running_in_gradescope():
    var = os.getenv(GRADESCOPE_ENV_VAR)
    if var is None:
        return False
    return var == 'yes'

## The Environment

For this assignment, we will use  `CartPole-v0` from OpenAI Gym. In this environment, the goal is to balance an inverted pendulum on a cart by moving the cart laterally. The state of the agent has four components:

- The horizontal position of the cart, $x$
- The velocity of the cart, $\dot{x}$
- The angle of the pendulum, measured relative to the vertical axis, $\theta$
- The angular velocity of the pendulum, $\dot\theta$

There are two actions, which apply leftward and rightward force to the cart.

The agent receives a reward of $1$ at each timestep,and the episode ends when the pendulum drops too far ($|\theta|$ gets large, around $\pi/2$ radians) or when the cart goes out of bounds. Also, the environment terminates after 200 steps if it hasn't already terminated, so the greatest possible return is $200$.

In [ ]:
env = gym.make('CartPole-v0')
env.reset()
cartpole_img = env.render(mode='rgb_array')
plt.imshow(cartpole_img)
env.close()

# Part 1. Parameterized Policy (30 pts total)

In this assignment, we will be studying Policy Gradient algorithms. In these algorithms, rather than using action-values to select actions, the policy itself is parameterized (in our case, by a neural network), and the policy is optimized directly via gradient ascent.

We will use a neural network to represent the policy here. The input to the neural network is a state, and the output should somehow encode a probability distribution over the action space. Our environment has a discrete action space, so the policy should output parameters for a *Categorical distribution*.



## 1a: The Policy Network (5 pts)
As a first step, fill in the `policy_init_network` function, which should return a torch neural net that will be to produce policy distributions for input states. You are free to experiment with different neural network architectures if you desire, but we recommend the following. Using `torch.nn.Sequential`, make a multilayer perceptron (MLP) with the following layers:

1. A linear layer of size `(state space dimension, 32)`, followed by a ReLU activation
1. A linear layer of size `(32, 32)`, followed by a ReLU activation
1. A linear layer of size `(32, number of actions)` followed by a Softmax activation, to make a probability distribution over actions.

In [ ]:
def policy_init_network(env: gym.Env) -> nn.Module:
  # ---------------------------------
  ...
  # ---------------------------------

In [ ]:
grader.check("question 1a")

## 1b: The Policy Class (15 pts)
In this part, we will build a class to represent the parameterized policy. This will be done in a few steps.

### Part I: The constructor
In the constructor of the `Policy` class, initialize the variable `opt`, which will be used to optimize the policy parameters. This variable should be a `torch.optim.Optimizer`.

### Part II: The policy distribution
Fill in the `dist` method. This method takes as input a state and outputs a torch [`Distribution`](https://pytorch.org/docs/stable/distributions.html) over actions.

### Part III: Sampling actions
Fill in the `action` method. This method samples a random action from the policy at a given state.

In [ ]:
class Policy:
  def __init__(self,
               env: gym.Env,
               network: nn.Module,
               discount=0.99,
               name="Abstract Policy"):
    self.name = name
    self.network = network
    self.discount = discount

    self.env = env
    self.obs_dim = env.observation_space.shape[0]
    self.n_actions = env.action_space.n

    # Your code here  (Part I)
    # ==========================
    ...
    # ==========================

  """
  Get the distribution over actions for a given state
  """
  def dist(self, x: np.ndarray) -> torchdist.distribution.Distribution:
    dist = None
    # Your code here (Part II)
    # ========================
    ...
    # ========================
    return dist

  """
  Sample a random action from the policy at a given state

  Input: a state encoded as a numpy array
  Output: an action encoded as an int
  """
  def action(self, x: np.ndarray) -> int:
    action = None
    # Your code here (Part III)
    # ========================
    ...
    # ========================
    return action

  def update(self, states, actions, rewards, dones) -> float:
    raise NotImplementedError


In [ ]:
grader.check("question 1b")

## 1c: Generating Rollouts (10 pts)

Now, fill in the following function that rolls out $N$ episodes in the environment with the policy. The function should return `(states, actions, rewards, dones)` where

1. `states` is a record of the states observed over the course of the episodes.
1. `actions` is a record of the actions taken.
1. `rewards` is a record of the rewards earned.
1. `dones` is an array of `bool`s that marks where the episodes end. For example, if each episode is 3 steps long, `dones = [False, False, True, False, False, True,...]`.

In [ ]:
def rollout(env: gym.Env, policy: Policy, N=1, render=False):
  states = []
  actions = []
  rewards = []
  dones = []
  # Your code here
  #==========================
  ...
  #==========================
  if render:
    env.close()
  return (np.array(states), np.array(actions), np.array(rewards), np.array(dones))

In [ ]:
grader.check("question 1c")

# 2. REINFORCE (47 pts total)
In this section, we will implement the REINFORCE algorithm.

<!-- BEGIN QUESTION -->


## 2a: Discounting Rewards (22 pts total)

This problem has 3 parts.

Recall the form of the REINFORCE policy gradient:

$$
\nabla_\theta J(\theta) = \sum_{k=0}^T\mathbf{E}\left\{G^{\pi_\theta}\nabla_\theta\log\pi_\theta(a_k\mid s_k)\right\}
$$

Here $\pi_\theta$ is the parameterized (neural net) policy with parameters $\theta$, and $G^{\pi_\theta}$ is the random variable corresponding to the discounted return induced by following $\pi_\theta$. Note that at timestep $k$, action $a_k$ had no influence on rewards incurred before timestep $k$. For this reason, it is generally preferred to compute the following,

$$
\widehat\nabla_\theta J(\theta) = \sum_{k=0}^T\mathbf{E}\left\{G^{\pi_\theta}_k\nabla_\theta\log\pi_\theta(a_k\mid s_k)\right\}
$$

where

$$
G^{\pi_\theta}_k = \sum_{t=k}^T\gamma^{t-k}r(s_k, a_k)
$$

### Part I (5 pts):
**Question**: Why do you think it is preferred to substitute $\nabla_\theta J(\theta)$ for $\widehat\nabla_\theta J(\theta)$ in policy gradient algorithms?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Part II (5 points):

It may help to make the assumption that at the end of an episode, the agent keeps receiving the final reward indefinitely. So, when an episode ends at time $T$, the "effective reward" at that last timestep is

$$
r_T = r(s_T, a_T) + \gamma r(s_T, a_T) + \gamma^2r(s_T, a_T) + \dots\\
= r(s_T, a_T)\left(\sum_{k=0}^\infty\gamma^k\right) 
$$

It's like as if the agent remains in its terminal state forever. When computing the discounted return of a trajectory, you may replace the terminal reward with this "effective reward". We will call this "the perpetuity trick".

**Question**: When should one apply the "perpetuity trick", and what effect would you expect it to have on training?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

### Part III (12 points):
Implement the function `discounted_returns`, which computes the values $(G_1^{\pi_\theta},G_2^{\pi_\theta},\dots)$ for a given sequence of rewards.

The function takes three arguments:

1. `rewards`: An array of rewards, which may have been collected over several trajectories. 
1. `dones`: An array of `bool`s, which mark where trajectories ended.
1. `discount`: The discount factor.
1. `perpetuity`: A boolean that indicates whether or not the perpetuity trick should be used.

The output of the function should be a list of the same length as `rewards` containing the cumulative discounted future returns starting at each step in the reward sequence. Mathematically, for some index $k$, if $T$ is the first index after $k$ for which `dones[T] = True`, then

$$
\texttt{returns}[k] = \texttt{rewards}[k] + \gamma\texttt{rewards}[k+1] + \dots + \gamma^{T-k}\texttt{rewards}[T]\quad \text{(+ perpetuity, if applicable)}
$$

For example, suppose we gather data from two trajectories, which had rewards `[1,2,3]` and `[4, 2, 1]` respectively. Then:

- `rewards = [1,2,3,4,2,1]`
- `dones = [False, False, True, False, False, True]`

For `discount = 0.5`, the output should be 

- Without perpetuity trick: `[2.75, 3.5, 3, 5.25, 2.5, 1]`.
- With perpetuity trick: `[3.5, 5, 6, 5.5, 3, 2]`.

**NOTE**: The output should be a numpy array.

In [ ]:
def discounted_returns(rewards: np.ndarray, dones: np.ndarray, discount:float, perpetuity = False) -> np.ndarray:
  # Your code here
  # ===============================
  ...
  # ===============================
  return returns

In [ ]:
grader.check("question 2a.3")

<!-- BEGIN QUESTION -->

## 2b: The REINFORCE Update (10 pts)
Finally, we'll implement REINFORCE. Fill in the `update` method for the `REINFORCEPolicy` class below. This method takes the following inputs:

1. `states`: An array of observed states over the course of $N$ episodes.
1. `actions`: An array of actions taken at the corresponding `states`.
1. `rewards`: An array of rewards received, where `rewards[k]` is the reward for taking actions `actions[k]` at state `states[k]`.
1. `dones`: An array of `bool`s marking the end of episodes.

This method should perform the following:
- Compute the average policy gradient "loss", which is $-\sum_{n=1}^{T}G_n^{\pi_\theta}\log\pi_\theta(a_n\mid s_n)$,  averaged over all trajectories
- Compute the policy gradient
- Update the policy parameters

The method should return a dictionary that contains information from the update. For now, the dictionary should only have one entry with key `'policy_loss'` that contains a scalar loss from the policy gradient computation.

In [ ]:
class REINFORCEPolicy(Policy):
  def __init__(self,
               env: gym.Env,
               network: nn.Module,
               discount=0.99,
               name="Plain REINFORCE"):
    super().__init__(env, network, discount=discount, name=name)
    
  """
  Perform a gradient update
  Inputs:
    states, actions, rewards, dones: Output from rollout method
  Returns:
    Dictionary with the following keys:
    - "policy_loss": float of the policy gradient loss (the quantity whose gradient is taken)
  """  
  def update(self, states, actions, rewards, dones) -> float:
    loss_dict = {}
    # Your code here
    # ======================
    ...
    # ======================
    return loss_dict

In [ ]:
grader.check("question 2b")

<!-- END QUESTION -->

## 2c: Experiments (15 pts)

The code below will train agents with REINFORCE using a varying number $N$ of trajectories per gradient estimate, for a fixed number of gradient steps. After each gradient step, we roll out 5 episodes and plot the mean and variance of their returns. Before running the experiments, think about how you expect the performance of the agents to depend on $N$.

In [ ]:
Ns = [1, 5, 10]
env = gym.make('CartPole-v0')
agents = [
    REINFORCEPolicy(env, policy_init_network(env))
    for _ in Ns
]

gradient_steps = 400
scores = [np.zeros(gradient_steps) for N in Ns]
stds = [np.zeros(gradient_steps) for N in Ns]

test_runs = 5

def rollout_score(env, policy):
  _, _, rewards, _ = rollout(env, policy, N=1)
  return np.sum(rewards)

gs = list(range(gradient_steps))

cmap = plt.get_cmap('viridis')
fig, (ret_ax, loss_ax) = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

if not running_in_gradescope():
    for i in range(len(Ns)):
      reinforce_policy = agents[i]
      N = Ns[i]
      print(f"N = {N}")
      losses = []
      for g in tqdm(range(gradient_steps)):
        states, actions, rewards, dones = rollout(env, reinforce_policy, N=N)
        loss = reinforce_policy.update(states, actions, rewards, dones)
        losses.append(loss['policy_loss'])
        res = [rollout_score(env, reinforce_policy) for _ in range(test_runs)]
        scores[i][g] = np.mean(res)
        stds[i][g] = np.std(res)
      color = cmap(i / len(Ns))
      ret_ax.plot(gs, scores[i], label=f"N={N}", color = color)
      ret_ax.fill_between(gs, scores[i] - stds[i], scores[i] + stds[i], alpha=0.3, color=color)
      loss_ax.plot(gs, losses, label=f"N={N}", color = color)
    ret_ax.legend()
    ret_ax.grid(True)
    ret_ax.margins(0)
    ret_ax.set_title('Episode return')
    loss_ax.legend()
    loss_ax.grid(True)
    loss_ax.margins(0)
    loss_ax.set_title("Policy loss")

In [ ]:
plt.close('all')

### Analysis
Based on your results, report how the number of rollout trajectories per update affected the learning process. Do your results match your expectations? Explain why or why not.

# 3. REINFORCE with Baseline (40 pts total)
When using a baseline in REINFORCE, the policy gradient formula is modified to the following,

$$
\nabla_\theta J(\theta) = \sum_{k=0}^T\mathbf{E}\left\{(G^{\pi_\theta} - b(s_k))\nabla_\theta\log\pi_\theta(a_k\mid s_k)\right\}
$$

for some function $b:\mathcal{S}\to\mathbf{R}$.

<!-- BEGIN QUESTION -->

## 3a: Understanding the baseline (12 pts)
1. **(3 pts)** What purpose does the baseline serve?
1. **(3 pts)** If the baseline is a constant (that is, $b(s_1) = b(s_2)$ for any pair of states $(s_1, s_2)$), should we expect the performance of REINFORCE with this baseline to be any different from standard REINFORCE?
1. **(3 pts)** Why can't the baseline be a function of the action as well as the state? 
1. **(3 pts)** Does the inclusion of an arbitrary baseline always help? 

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## 3b: The Value Function (3 pts)
In our experiments, we will use the value function as our baseline. It will be necessary to learn the value function from data, so our baseline will have the form

$$
b(s) = V^{\pi_\theta}_\phi(s)
$$

where $\phi$ denotes the parameters of the value function.

Fill in the code for the construction of the value function neural net in `value_init_network`. The network architecture should be similar to that of the policy network besides the output layer.

In [ ]:
def value_init_network(env: gym.Env) -> nn.Module:
  # Your code here
  # ===========================================================
  ...
  # ===========================================================

In [ ]:
grader.check("question 3b")

<!-- BEGIN QUESTION -->

## 3c: REINFORCE with Baseline (10 pts)

Fill in the constructor and the `update` method for `REINFORCEWithBaselinePolicy`.

The constructor should do set two variables:
* `self.value_network`: the value function neural network
* `self.value_opt`: the `torch.optim.Optimizer` for the value function parameters

This method should perform the following:
- Compute the "policy gradient loss", using the value predictions from the value function network instead of the Monte Carlo return estimates
- Compute the policy gradient, again using the value predictions from the value function network instead of the Monte Carlo return estimates
- Update the policy parameters
- Compute the "value loss", which is mean squared difference between the Monte Carlo return estimates and the value function network predictions at each state in the trajectory
- Update the value function network parameters

As with the standard REINFORCE case, the `update` method returns a dictionary with a key `'policy_loss'` reflecting the loss w.r.t. the policy gradient objective. For `REINFORCEWithBaselinePolicy`'s `update` method, however, the dictionary should also have a key `'value_loss'` reflecting the loss w.r.t. the value function error.

In [ ]:
class REINFORCEWithBaselinePolicy(Policy):
  def __init__(self,
               env: gym.Env,
               policy_network: nn.Module,
               value_network: nn.Module,
               discount=0.99,
               name="REINFORCE with Baseline"):
    super().__init__(env, policy_network, discount=discount, name=name)
    # Your code here
    # ===========================================================
    ...
    # ===========================================================
  
  """
  Perform a gradient update
  Inputs:
    states, actions, rewards, dones: Output from rollout method
  Returns:
    Dictionary with the following keys:
    - "policy_loss": float of the policy gradient loss (the quantity whose gradient is taken)
    - "value_loss": float of the squared TD error
  """  
  def update(self, states, actions, rewards, dones) -> float:
    loss_dict = {}
    # Your code here
    # ======================
    ...
    # ======================
    return loss_dict

In [ ]:
grader.check("question 3c")

<!-- END QUESTION -->

## 3d: Experiments (15 pts)

The code below will train agents with REINFORCE with and without the value function baseline. Think about how you expect the return and loss curves to behave with and without the baseline.

In [ ]:
agents = [
    REINFORCEPolicy(env, policy_init_network(env)),
    REINFORCEWithBaselinePolicy(env, policy_init_network(env), value_init_network(env))
]

gradient_steps = 400
scores = [np.zeros(gradient_steps) for _ in agents]
stds = [np.zeros(gradient_steps) for _ in agents]

test_runs = 5

def rollout_score(env, policy):
  _, _, rewards, _ = rollout(env, policy, N=1)
  return np.sum(rewards)

gs = list(range(gradient_steps))

cmap = plt.get_cmap('viridis')
fig, (ret_ax, loss_ax, value_ax) = plt.subplots(nrows=1, ncols=3, figsize=(18, 6))

value_losses = []

if not running_in_gradescope():
    for i in range(len(agents)):
      reinforce_policy = agents[i]
      print(f"Training {reinforce_policy.name}")
      losses = []
      for g in tqdm(range(gradient_steps)):
        states, actions, rewards, dones = rollout(env, reinforce_policy, N=1)
        loss = reinforce_policy.update(states, actions, rewards, dones)
        losses.append(loss['policy_loss'])
        if 'value_loss' in loss.keys():
          value_losses.append(loss['value_loss'])
        res = [rollout_score(env, reinforce_policy) for _ in range(test_runs)]
        scores[i][g] = np.mean(res)
        stds[i][g] = np.std(res)
      color = cmap(i / len(agents))
      ret_ax.plot(gs, scores[i], label=reinforce_policy.name, color = color)
      ret_ax.fill_between(gs, scores[i] - stds[i], scores[i] + stds[i], alpha=0.3, color=color)
      loss_ax.plot(gs, losses, label=reinforce_policy.name, color = color)
    ret_ax.legend()
    ret_ax.grid(True)
    ret_ax.margins(0)
    ret_ax.set_title('Episode return')
    loss_ax.legend()
    loss_ax.grid(True)
    loss_ax.margins(0)
    loss_ax.set_title("Policy loss")
    value_ax.plot(gs, value_losses, color = color)
    value_ax.grid(True)
    value_ax.margins(0)
    value_ax.set_title("Value loss")

In [ ]:
plt.close('all')

### Analysis
In your experiments, how did the use of the value function baseline affect your results? Explain the results you observed.

# 4. Actor-Critic (50 pts total)

Finally, we will experiment with an *actor-critic* algorithm. Recall that the gradient rule for REINFORCE with the value function baseline has the following form,

$$
\nabla_\theta J(\theta) = \sum_{k=0}^T\mathbf{E}\left\{(G^{\pi_\theta} - V^{\pi_\theta}_\phi(s_k))\nabla_\theta\log\pi_\theta(a_k\mid s_k)\right\}
$$

Note that

$$
\mathbf{E}\left\{G^{\pi_\theta}\mid s_0 = s\right\} = \mathbf{E}_{a\sim\pi(\cdot\mid s),s'\sim P(\cdot\mid s, a)}\left\{r(s, a) + \gamma V^{\pi_\theta}(s')\right\}
$$

Because of this, actor-critic algorithms estimate $G^{\pi_\theta}$ by $r(s, a) + \gamma V^{\pi_\theta}(s')$. Thus, we can compute one gradient *per environment step*, since we no longer need data from the entire trajectory to estimate $G^{\pi_\theta}$. The gradient rule for the policy network (actor) is

$$
\nabla_\theta J_{\text{actor}}(\theta) = \mathbf{E}\left\{(r_k + \gamma V^{\pi_\theta}_\phi(s_{k+1}) - V^{\pi_\theta}_\phi(s_k))\nabla_\theta\log\pi_\theta(a_k\mid s_k)\right\}
$$

for the policy parameters. The value network (critic) is trained to minimize the mean squared TD error:

$$
\nabla_\phi J_{\text{critic}}(\phi) = \frac{1}{2}\left(V^{\pi_\theta}_\phi(s_k) - \texttt{stop_gradient}\left(r_k + \gamma V^{\pi_\theta}_\phi(s_{k+1})\right)\right)^2
$$

where $\texttt{stop_gradient}$ enforces that no gradients flow through its argument.

## 4a: Understanding Actor-Critic (15 pts total)

This question is split into three conceptual questions.

<!-- BEGIN QUESTION -->

### Part I: Bias in Actor-Critic (5 pts)
It is said that actor-critic policy gradients are more biased than REINFORCE policy gradients. Explain what this means. Are actor-critic policy gradients more biased than REINFORCE policy gradients computed with the value function baseline?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Part II: Per-Step Updates (5 pts)
Even though actor-critic algorithms can perform one update per step, the gradients are computed based on data from only one state transition as opposed to REINFORCE gradients which are averaged over $T$ state transitions. What is the benefit of updating once per environment step?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Part III: Lifelong Learning (5 pts)
Imagine a scenario where an RL agent is to be deployed on a strange planet that we do not know how to simulate. Once we drop the robot on this planet, we can never interact with it again: it just autonomously learns from environment interactions for the rest of its life. Would you prefer to employ Actor-Critic or REINFORCE with baseline for this problem? Why?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## 4b: Implementing Actor-Critic (20 pts)
Fill out the `ActorCriticPolicy` class below, according to the guidelines in the code. The `policy_init_network` and `value_init_network` methods will be used to instantiate the neural nets for the actor-critic, however they are trained differently in the actor-critic algorithm.

Rather than implementing an `update` method for actor-critic, we will implement a method `train_episode` which rolls out an episode, performing updates at each step. More precisely, `train_episode` should do the following:

1. Reset the environment to a starting state
1. For each environment step:
    1. Choose an action
    1. Perform an environment step with the chosen action, observing the next state, reward, and terminal signal
    1. Update both the actor and critic networks based on this transition
1. Return a dictionary with the same entries as `REINFORCEWithBaselinePolicy`'s `update` method.

In [ ]:
class ActorCriticPolicy(Policy):
  def __init__(self,
               env: gym.Env,
               policy_network: nn.Module,
               value_network: nn.Module,
               discount=0.99,
               name="Actor-Critic"):
    super().__init__(env, policy_network, discount=discount, name=name)
    # Your code here
    # Initialize self.value_network and self.value_opt like before
    # ===========================================================
    ...
    # ===========================================================
  
  """
  Run a training episode
  Inputs:
    states, actions, rewards, dones: Output from rollout method
  Returns:
    Dictionary with the following keys:
    - "policy_loss": float of the policy gradient loss (the quantity whose gradient is taken),
                     averaged over the episode
    - "value_loss": float of the squared TD error averaged over the episode
  """  
  def train_episode(self) -> float:
    loss_dict = {}
    # Your code here
    # ======================
    ...
    # ======================
    return loss_dict

In [ ]:
grader.check("question 4b")

<!-- END QUESTION -->

## 4c: Experiments (15 pts)

In the following experiments, we test the following agents:

- REINFORCE with one trajectory per gradient update
- REINFORCE with the value function baseline, one trajectory per gradient update
- Actor-Critic

Each agent is trained for 400 episodes, and the experiment is repeated 6 times with different random seeds. The plot displays the mean and variance of the return across the seeds for each agent.

In [ ]:
import itertools
SEEDS = [4, 8, 15, 16, 23, 42]

episodes = 400
eval_runs = 5
eval_every = 5
epochs = list(range(0, episodes, eval_every))

cmap = plt.get_cmap('viridis')
plt.grid(True)
plt.margins(0)
plt.xlabel("Episode")
plt.ylabel("Return")

pg_constructor = lambda: REINFORCEPolicy(env, policy_init_network(env))
pg_baseline_constructor = lambda: REINFORCEWithBaselinePolicy(env, policy_init_network(env), value_init_network(env))

ac_agent_constructor = lambda: ActorCriticPolicy(env, policy_init_network(env), value_init_network(env))

if not running_in_gradescope():
    ### ACTOR CRITIC
    ac_agents = {}
    ac_score_traces = {}
    print(f"Training Actor-Critic")
    for (seed, ep) in tqdm(itertools.product(SEEDS, np.arange(episodes))):
        if seed not in ac_agents.keys():
            np.random.seed(seed)
            torch.manual_seed(seed)
            ac_agents[seed] = ac_agent_constructor()
            ac_score_traces[seed] = []
            ac_agents[seed].env.seed(seed)
        agent = ac_agents[seed]
        loss = agent.train_episode()
        if (ep + 1) % eval_every == 0:
            res = [rollout_score(env, agent) for _ in range(eval_runs)]
            ac_score_traces[seed].append(np.mean(res))

    ac_data = np.vstack([ac_score_traces[seed] for seed in SEEDS])
    ac_score_mean = np.mean(ac_data, axis=0)
    ac_score_std = np.std(ac_data, axis=0)

    plt.plot(epochs, ac_score_mean, color=cmap(0.8), label='Actor Critic')
    plt.fill_between(
        epochs,
        ac_score_mean - ac_score_std,
        ac_score_mean + ac_score_std,
        color=cmap(0.8),
        alpha=0.3
    )

    ### REINFORCE WITH BASELINE
    pg_baseline_agents = {}
    pg_baseline_score_traces = {}
    print(f"Training REINFORCE with Baseline")
    for (seed, ep) in tqdm(itertools.product(SEEDS, np.arange(episodes))):
        if seed not in pg_baseline_agents.keys():
            np.random.seed(seed)
            torch.manual_seed(seed)
            pg_baseline_agents[seed] = pg_baseline_constructor()
            pg_baseline_score_traces[seed] = []
            env.seed(seed)
        agent = pg_baseline_agents[seed]
        states, actions, rewards, dones = rollout(env, agent, N=1)
        loss = agent.update(states, actions, rewards, dones)
        if (ep + 1) % eval_every == 0:
            res = [rollout_score(env, agent) for _ in range(eval_runs)]
            pg_baseline_score_traces[seed].append(np.mean(res))

    pg_baseline_data = np.vstack([pg_baseline_score_traces[seed] for seed in SEEDS])
    pg_baseline_score_mean = np.mean(pg_baseline_data, axis=0)
    pg_baseline_score_std = np.std(pg_baseline_data, axis=0)

    plt.plot(epochs, pg_baseline_score_mean, color=cmap(0.5), label='REINFORCE with Baseline')
    plt.fill_between(
        epochs,
        pg_baseline_score_mean - pg_baseline_score_std,
        pg_baseline_score_mean + pg_baseline_score_std,
        color=cmap(0.5),
        alpha=0.3
    )

    ### REINFORCE
    pg_agents = {}
    pg_score_traces = {}
    print(f"Training REINFORCE with Baseline")
    for (seed, ep) in tqdm(itertools.product(SEEDS, np.arange(episodes))):
        if seed not in pg_agents.keys():
            np.random.seed(seed)
            torch.manual_seed(seed)
            pg_agents[seed] = pg_constructor()
            pg_score_traces[seed] = []
            env.seed(seed)
        agent = pg_agents[seed]
        states, actions, rewards, dones = rollout(env, agent, N=1)
        loss = agent.update(states, actions, rewards, dones)
        if (ep + 1) % eval_every == 0:
            res = [rollout_score(env, agent) for _ in range(eval_runs)]
            pg_score_traces[seed].append(np.mean(res))

    pg_data = np.vstack([pg_score_traces[seed] for seed in SEEDS])
    pg_score_mean = np.mean(pg_data, axis=0)
    pg_score_std = np.std(pg_data, axis=0)

    plt.plot(epochs, pg_score_mean, color=cmap(0.2), label='REINFORCE')
    plt.fill_between(
        epochs,
        pg_score_mean - pg_score_std,
        pg_score_mean + pg_score_std,
        color=cmap(0.2),
        alpha=0.3
    )

    plt.legend()

In [ ]:
plt.close('all')

### Analysis
Based on your experiments, does actor-critic perform favorably to REINFORCE (with and/or without baseline)? Explain your observations. 